In [ ]:
#Average FRE and F-K scores per year per author
single_avg = (
    long_df.groupby(['Year', 'Author(s)'])
    .agg(
        FRE_Score=('FRE Score', 'mean'),
        FK_Score=('F-K Score', 'mean')
    )
    .reset_index()
)

justice_selection = alt.selection_point(fields=['Author(s)'])

#Nearest tooltip
nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Year', 'Author(s)'],
    empty='none'
)

#Highlight
highlight_color = alt.condition(
    justice_selection,
    alt.Color('Author(s):N', legend=None, scale=alt.Scale(scheme='category10')),
    alt.value('lightgray')
)

#FRE Chart
line_chart_fre = alt.Chart(single_avg).mark_line().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('FRE_Score:Q', title='FRE Score', scale=alt.Scale(domain=[35, 75])),
    color=highlight_color
).properties(
    title='Line Chart of FRE Scores by Author Over Time',
    width=600,
    height=400
).add_params(justice_selection)

#FRE tooltips
fre_tooltip = alt.Chart(single_avg).mark_point(size=50, opacity=0).encode(
    x='Year:O',
    y='FRE_Score:Q',
    tooltip=[
        'Year:O', 
        'FRE_Score:Q', 
        'Author(s):N'
    ]
).add_params(nearest)

fre_combined_chart = line_chart_fre + fre_tooltip

#FRE legend
fre_legend = alt.Chart(single_avg).mark_point().encode(
    y=alt.Y('Author(s):N', axis=alt.Axis(title='Author(s)', labelLimit=100)),
    color=highlight_color
).properties(
    width=50
).add_params(justice_selection)

#F-K Chart
line_chart_fk = alt.Chart(single_avg).mark_line().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('FK_Score:Q', title='F-K Score', scale=alt.Scale(domain=[4, 12])),
    color=highlight_color
).properties(
    title='Line Chart of F-K Scores by Author Over Time',
    width=600,
    height=400
).add_params(justice_selection)

#F-K tooltip
fk_tooltip = alt.Chart(single_avg).mark_point(size=50, opacity=0).encode(
    x='Year:O',
    y='FK_Score:Q',
    tooltip=[
        'Year:O', 
        'FK_Score:Q', 
        'Author(s):N'
    ]
).add_params(nearest)

fk_combined_chart = line_chart_fk + fk_tooltip

#F-K legend
fk_legend = alt.Chart(single_avg).mark_point().encode(
    y=alt.Y('Author(s):N', axis=alt.Axis(title='Author(s)', labelLimit=100)),
    color=highlight_color
).properties(
    width=50
).add_params(justice_selection)

#Combine
fre_combined = alt.hconcat(fre_combined_chart, fre_legend).resolve_scale(color='independent')
fk_combined = alt.hconcat(fk_combined_chart, fk_legend).resolve_scale(color='independent')

line_charts = alt.vconcat(fre_combined, fk_combined).configure_view(
    stroke=None
)

#Display
line_charts